下文是从一个知乎笔友的笔记中按照文档代码复现学习了一遍，干货满满，知乎原文链接：https://zhuanlan.zhihu.com/p/626819858
，通过这次学习对近期研究AutoGPT有了更深的理解。

实际课程是吴恩达老师近期和openai合作推出的课程《ChatGPT Prompt Engineering for Developers》，链接见：https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

课程大纲：
- 1 介绍
- 2 指引
- 3 使用迭代开发策略改进你的提示语
- 4 文本摘要
- 5 推断任务
- 6 文本转换
- 7 扩展文本
- 8 创建聊天机器人
- 9 总结

我个人笔记中，将第8点放在了最前面，会先介绍一下ChatGPT API 的消息机制，有助于后续的理解。

In [ ]:
import openai

# 改为自己的openai key
openai.api_key = "XXX"

# 通用方法，后面会用到
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# 创建聊天机器人
## ChatGPT API 的消息机制
### 三种对话角色的作用



在我们上面定义的工具函数中，每条提示语并是不仅仅简单地以字符串形式被发送到了 OpenAI 的服务器上，而是被包装在了一个名为 messages 的字典列表中。并且我们的提示还拥有一个角色属性“user”，这与 ChatGPT 的三种角色的设计有关。

我们与 ChatGPT 的所有对话均被分成三种角色。"system" 系统角色，即对话的底层设定，包括模型应该扮演怎样的角色、给用户何种类型的反馈等等，这类角色通常在对话开始之前由后台给出，并不暴露给用户。此外还有 "user" 用户角色，用户向模型发送的信息使用的角色，“assistant”助理角色，即对话过程中模型输出使用的角色。



In [16]:
# 例子：一段由"system"角色给出基本设定的对话
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


response = get_completion_from_messages(messages, temperature=1)
print(response)


To get to the other side! Haha! Verily, 'tis an old one but still fair.


cc：注意：用户与聊天机器人的交互是开发者无法掌控的。在构建聊天机器人的任务中，我们对模型的控制主要在于以"system"角色给出的底层设定。


### 维护 messages 列表

一个稍有些违反直觉的事实是，尽管模型显得能够记住我们之前说过的话，但事实上我们每一次对模型的输入都是一次全新的请求，模型并不会将之前的对话以某种方式“记忆”在权重里。我们之所以能够实现连续对话，是因为网站后台会自动将之前的对话内容保存下来，在每次请求时把这些的记录作为前文一并发送给模型，而每句对话的角色属性则是用于区分哪句话是用户说的、哪句话是模型自己说的、哪句话是系统设定的“标签”。



In [17]:
# 例子：把对话内容分两次单独放进 messages 列表发送，模型“遗忘”了之前的对话

messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?
I'm sorry, but as a chatbot, I don't have access to that information. Could you please tell me your name?


In [18]:
# 例子：把完整的对话记录放进 messages 里给出，实现了与模型的连续对话
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!


In [20]:
# 例子：实现一个披萨店自助点餐机器人

import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

# 输入框获取输入信息
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')


def collect_messages(_): # 历史对话构建函数
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

# 发送信息按钮
button_conversation = pn.widgets.Button(name="Chat!")

# 列表展示对话
interactive_conversation = pn.bind(collect_messages, button_conversation)

# 界面函数
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

# 在 jupyternote book 中展示界面
dashboard



Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [21]:
# 用"system"角色提示模型生成此次点单的账单：
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)


Sure, here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "Pepperoni",
    "size": "Small",
    "price": 12.95
  },
  "toppings": [],
  "drinks": [
    {
      "type": "Coke",
      "size": "Small",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 15.95
}
```

Let me know if you have any questions!


# 指引

## 使用提示语的两个原则
在本节课中，吴恩达老师给出了使用提示语的两个原则：

1. 使用明确且具体的指令（Write clear and specific instructions）

2. 给模型充足的“思考”时间 （Give the model time to “think”） 我们将首先在较高层次上审视它们，然后通过示例展示对应于这两条原则的具体策略。

### 原则一：使用明确且具体的指令
应当通过提供尽可能明确和具体的指令来表达我们希望模型执行的操作。这将引导模型获得所需的输出，并减少获得不相关或不正确响应的可能性。 **注意不要混淆“明确的提示”和“简短的提示”**，因为在许多情况下，较长的提示实际上为模型提供了更多的清晰度和上下文，这实际上可以带来更详细和相关的输出。

#### **策略一： 使用定界符清楚地指示输入的不同部分**

定界符可以是任何明确的标点符号，将需要处理的的文本片段与指令部分分开，让模型非常清楚它应该处理的确切文本。

- 定界符可以使用任何符号，例如: ''', """, < >, <tag> </tag>, :

- 常用定界符的英文称呼

- Triple quotes: """

- Triple backticks: ```

- Triple dashes: ---

- Angle brackets: < >,

- XML tags: <tag> </tag>



例子：提示模型对被```包裹的一段文字给出摘要

In [15]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
\`\`\`{text}\`\`\`
"""
# 使用定界符限制文本部分能够让模型清晰地区分开指令和需要处理的文本，以避免提示注入现象的发生。


response = get_completion(prompt)
print (response)


Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.


#### **策略二：请求结构化的输出**

为了更轻松地解析模型的输出，我们可以请求模型以 HTML 或 JSON 这样的结构化格式输出信息。

例子：以 JSON 格式生成一个虚构的书目列表



In [14]:
prompt2 = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt2)
print(response)

[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret Life of Bees",
    "author": "Lila Rose",
    "genre": "Romance"
  }
]


#### **策略三：让模型检查条件是否满足**

如果执行一项任务需要满足一定的前提条件，那么我们可以告诉模型首先检查这些条件是否被满足，只有满足时才执行任务，不满足时则输出提示信息。

例子：如果文本中含有一系列指令则按一定格式输出，否则输出提示信息

In [16]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!




In [17]:
# 这是一个反例
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)



Completion for Text 2:
No steps provided.


#### **策略四：“少样本”提示**

“少样本”（few-shot）可以理解为模型利用少量未知领域的样本举一反三的能力。当我们对模型的输出有一个比较明确的预期时，我们可以通过举一些例子告诉模型我们期望得到怎样的输出。之后模型将会根据我们给出的少量样本执行更多的任务。

例子：给出例子，教模型用外婆的语气回答孩子的问题

In [18]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


### 原则二：留给模型充足的“思考”时间

正如我们在第一课所学，ChatGPT 建立在基本大语言模型的基础上，其原理依旧是依据前文不断预测下一个输出的单词。

所以，我们有两种方式让它得出结论：
1. 仅通过用户的问题直接逐个单词预测答案；
2. 先让模型输出一段思考过程，再根据问题和思考过程作为前文预测答案。类比于我们解复杂的数学题总是要一步一步做一样，让模型先输出一段思考过程的第二种方式效果往往都会优于直接得出答案的第一种方式。因此留给模型充足的“思考时间”是一项很重要的原则。



#### **策略一：让模型分步骤完成任务**

例子：按特定步骤处理一段文字

In [19]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
\`\`\`{text}\`\`\`
"""
response = get_completion (prompt_1)
print ("Completion for prompt 1: ")
print (response)

Completion for prompt 1: 
Two siblings go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

两个兄弟姐妹前往山顶的井取水，但不幸发生，他们滚下山，虽然有些轻伤，但他们的冒险精神仍然不减，继续愉快地探索。 
兄弟姐妹 
{"chinese_summary": "两个兄弟姐妹前往山顶的井取水，但不幸发生，他们滚下山，虽然有些轻伤，但他们的冒险精神仍然不减，继续愉快地探索。", "num_names": 2}


In [20]:
# cc：上面对中文的支持还是不够好，后面换成gpt-4再试试。
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
\`\`\`{text}\`\`\`
"""
response = get_completion (prompt_1)
print ("Completion for prompt 1: ")
print (response)

Completion for prompt 1: 
Two siblings, Jack and Jill, go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits au sommet d'une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts. 

Names: Jack, Jill.

{
  "french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits au sommet d'une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.",
  "num_names": 2
}


例子：在前面的基础上令模型按指定格式输出结果

In [22]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed. 
Translation: Jack et Jill partent en quête d'eau, mais la malchance frappe et ils dégringolent la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill partent en quête d'eau, mais la malchance frappe et ils dégringolent la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.", "num_names": 2}


#### 策略二：让模型先给出自己的解决方案，再下结论

例子：给出一道题和一段解答，判断解答是否正确 让模型直接判断正误：


In [27]:
# 这个案例中，学生的回答是错误的，但是模型将错误的题解判断为了正确的。
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


In [28]:
# 这个案例 让模型先自己给一个题解，再判断我们给的题解是否正确：
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
\`\`\`
question here
\`\`\`
Student's solution:
\`\`\`
student's solution here
\`\`\`
Actual solution:
\`\`\`
steps to work out the solution and your solution here
\`\`\`
Is the student's solution the same as actual solution \
just calculated:
\`\`\`
yes or no
\`\`\`
Student grade:
\`\`\`
correct or incorrect
\`\`\`

Question:
\`\`\`
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
\`\`\`
Student's solution:
\`\`\`
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100, 000 + 100x
Total cost: 100x + 250x + 100, 000 + 100x = 450x + 100, 000
\`\`\`
Actual solution:
"""
response = get_completion(prompt)
print(response)



```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


cc：在论文《Large Language Models are Zero-Shot Reasoners》中，作者发现只要在提示语中里加上一句“Let's think step by step”就能提高 Chatgpt 回答的逻辑性，这一方法也被称为「思维链」(Chain of Thought, CoT)。

### 模型的局限性：幻觉 (Hallucinations)

ChatGPT 的一项局限性在于，它并不十分清楚自己知识的边界在哪里。这意味着它可能会在回答某些话题时编造一些看似合理实则不正确的信息（口胡）。这种模型虚构的信息被称作“幻觉”（Hallucinations）。 为了避免模型幻觉，我们首先应当确保自己的提示里不包含误导性的错误信息；其次避免让模型回答一些生僻的或前置条件不够的知识性问题，而更多让模型基于我们的文本给出回答；最后，为了避免模型输出偏离我们给出的文本，我们可以要求模型在生成回答的同时引用原文以提高准确性。

其他资料：
一种通过标号实现模型输出引用原文的方法：https://www.playpcesor.com/2023/03/chatgpt-2.html

例子： 提示中含有虚构的产品

In [29]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
""" # Boie 是一家现实中的公司，但这里提到的产品是虚构的
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. It features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the most stubborn stains and buildup, and it can help you achieve a brighter, healthier smile in just a few weeks.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're brushing too hard.



# 使用迭代开发策略改进你的提示语

## 提示应用的开发理论

当我们为了解决任务构造合适的提示语时，我们实际上已经处在开发一个提示应用（prompt application）的过程中。在这个过程中，我们往往不能一次性找到最合适的那个提示语，而需要根据模型的输出不断调整和修改。我们可以借鉴软件工程的思想，用迭代策略（iterative strategy）系统化我们改进提示语的过程，而这和大语言模型开发的迭代过程是非常相进的。

大语言模型的迭代开发过程：点子->实现->实验结果->错误分析->回到点子->...

提示应用的开发过程：

1. 给出明确且具体的指令； 
2. 查看模型的输出； 
3. 分析为什么没有得到想要的输出，是因为指令不够明确，还是没有给模型足够的思考时间？ 
4. 改进思路和提示语，进一步明确指令，给模型更多思考时间。 
5. 回到第一步继续，直到得到满意的结果。

例子：从产品说明书生成营销文案


In [31]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""


## 第一轮迭代：使用基本的提示生成营销文案

In [32]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: \`\`\`{fact_sheet_chair}\`\`\`
"""
response = get_completion (prompt)
print (response)

Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. Part of a beautiful family of office furniture, this chair comes in several options of shell color and base finishes, including stainless steel, matte black, gloss white, or chrome. You can choose between plastic back and front upholstery or full upholstery in 10 fabric and 6 leather options. The chair is also available with or without armrests, and is qualified for contract use.

Constructed with a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action, this chair is both sturdy and comfortable. The dimensions of the chair are WIDTH 53 CM | 20.87”, DEPTH 51 CM | 20.08”, HEIGHT 80 CM | 31.50”, SEAT HEIGHT 44 CM | 17.32”, and SEAT DEPTH 41 CM | 16.14”. You can also choose between soft or hard-floor caster options, two choices of seat foam densities, and armless or 8 position PU armrests.

The materials used in the construction of this cha

- 实际效果：忠实地反映了产品说明书的内容。

- 存在的问题：输出的文案过长，不符合作为营销文案。

- 分析原因：在提示语中没有明确地限制输出长度。



## 第二轮迭代：一段带有长度限制的提示

In [33]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: \`\`\`{fact_sheet_chair}\`\`\`
""" # 注：除了通过限定单词数以外，还可以限制句子数甚至字符数。
response = get_completion (prompt)
print (response)

Introducing the mid-century inspired office chair, part of a beautiful furniture family. Available in various shell colors and base finishes, with plastic or full upholstery options. Suitable for home or business use, with pneumatic chair adjust and soft/hard-floor caster options. Made in Italy with high-quality materials.


- 实际效果：忠实反映了产品主要特点，字数合适。

- 存在的问题：如果我们是销售部门，那么我们会希望营销的文案更加具有针对性，能够抓住特定受众的喜好。因此我们以目标用户为家具零售商为例继续改进我们的提示。

- 分析原因：文案并未聚焦于目标受众的关注点——技术参数。



## 第三轮迭代：关注于特定受众偏好的提示语

In [35]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: \`\`\`{fact_sheet_chair}\`\`\`
"""
response = get_completion (prompt)
print (response)

Introducing the mid-century inspired office chair, perfect for home or business settings. With a range of shell colors and base finishes, including stainless steel and matte black, this chair is available with or without armrests and in 10 fabric and 6 leather options. The 5-wheel plastic coated aluminum base and pneumatic chair adjust make for easy movement and height adjustment. Every 7-character Product ID: SWC-100, SWC-110.


- 实际效果：更加关注技术细节，满足了对目标客户的针对性，并且遵循了格式要求。

- 存在的问题：项目经理告诉你需求改了，新的销售文案要 HTML 格式的，还要带一份有关产品的零件尺寸表。结果这个提示语也不符合要求了。

- 分析原因：提示语中没有对格式和零件尺寸表的具体要求。



## 最后一轮迭代：明确要求长度限制、关注点、输出格式和额外信息的提示语


In [36]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: \`\`\`{fact_sheet_chair}\`\`\`
"""

response = get_completion (prompt)
print (response)

<div>
<p>Introducing our mid-century inspired office chair, perfect for home or business settings. Available in a variety of shell colors and base finishes, with plastic or full upholstery options in fabric or leather. The chair features a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. Product ID: SWC-100, SWC-110.</p>
</div>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Width</th>
    <td>53 cm | 20.87"</td>
  </tr>
  <tr>
    <th>Depth</th>
    <td>51 cm | 20.08"</td>
  </tr>
  <tr>
    <th>Height</th>
    <td>80 cm | 31.50"</td>
  </tr>
  <tr>
    <th>Seat Height</th>
    <td>44 cm | 17.32"</td>
  </tr>
  <tr>
    <th>Seat Depth</th>
    <td>41 cm | 16.14"</td>
  </tr>
</table>


在如上的例子中，我们使用了一段产品说明书来评估提示的好坏，进而分析提示不足的原因展开迭代。但对于一些复杂的应用程序，则往往需要通过一个批次（10 个甚至 50 个以上）的样本评估，才能选出最终效果最佳的提示语。不过，一般而言也只有在大型提示应用开发的最后几个步骤才会用到大量的样本来改进提示效果，而对于个人用户而言，很多提示语是都是一次性的，只在自己的文本上进行评估迭代就足够了。

cc：这里就牵扯到了，我们在设计复杂应该的时候应该用什么样的提示语？

# 文本摘要

摘要或是 ChatGPT 的一项非常实用的应用，它可以帮助我们快速从长文本中找到自己需要的信息，而无需花时间亲自读完整段文本，从而大大加快我们我们查资料、了解信息的过程。 在商业中，文本摘要技术也存在广泛的应用。以下是一段客户对商品的评价，我们将代替商家总结出这篇评价中的主要部分，以便于商家快速浏览这些评价，找出商品在市场上的优势和不足之处。

In [38]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

## 使用基本的提示生成摘要

In [40]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: \`\`\`{prod_review}\`\`\`
"""

response = get_completion (prompt)
print (response)

Soft and cute panda plush toy with a friendly face. A bit small for the price, but arrived a day early. Daughter loves it.


## 如何生成有侧重点的摘要

如果商家希望把摘要反馈给快递部门，那么就要生成的摘需要更加侧重于商品运输和派送过程方面。接下来我们修改提示语来实现这一点：


In [41]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: \`\`\`{prod_review}\`\`\`
"""

response = get_completion (prompt)
print (response)

The panda plush toy arrived a day earlier than expected, but the customer felt it was a bit small for the price paid.


In [42]:
# 如果希望摘要更关注商品定价方面，则可以将提示修改为：
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: \`\`\`{prod_review}\`\`\`
"""

response = get_completion (prompt)
print (response)

The panda plush toy is soft, cute, and loved by the recipient, but the price may be too high for its size compared to other options.


## 从“生成摘要”到“提取信息”
在刚才的例子中，我们尝试生成了关注于某一些方面的摘要。如果我们希望更进一步，只保留我们关注的方面而不要其他信息，则可以将摘要任务转换为提取信息的任务：

In [43]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: \`\`\`{prod_review}\`\`\`
"""

response = get_completion (prompt)
print (response)

"The product arrived a day earlier than expected." - Feedback for Shipping department: Good job on delivering the product earlier than expected.


## 批量生成文本摘要

In [54]:
review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

from time import sleep

reviews = [review_1, review_2, review_3, review_4]
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: \`\`\`{reviews[i]}\`\`\`
    """

    response = get_completion(prompt)
    print(i, response, "\n")
    sleep(20)

0 Soft and cute panda plush toy loved by daughter, but a bit small for the price. Arrived early. 

1 Affordable lamp with storage, fast shipping, and excellent customer service. Easy to assemble and missing parts were quickly replaced. 

2 Good battery life, small toothbrush head, good deal if bought around $50. Makes teeth feel clean. 

3 The product was on sale for $49 in November, but the price increased to $70-$89 in December. The base doesn't look as good as previous editions, but works well for crushing hard items. Freeze fruits and vegetables for smoothies to avoid adding ice. The motor made a funny noise after a year and the warranty had expired. Overall quality has decreased. 



# 推断任务

情感分析、实体识别与主题提取是三种常见的推断工作。以往的深度学习工程师如果需要实现这三个任务，则至少要用不同的数据集把模型训练为三种不同的权重。而 ChatGPT 搭配合适的提示语把这些曾经麻烦的任务变得唾手可得。

我们以下面的灯具商品评价，和一段待处理的故事文本为例介绍如何利用 ChatGPT 实现情感分析、实体识别与主题提取这三种推断工作：

In [55]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## 情感分析任务

情感分析是指由模型推断一句话的情感倾向（正面/中立/负面）或所包含的情感（欢快、愤怒等），具有非常广泛的应用。在下面的例子中，我们通过编写合适的提示语让 ChatGPT 为我们分析商品评价的情感色彩，进而可以把这些评价按消极/积极归类起来，留作不同用途。



In [58]:
# 例子：判断商品评价的整体情感倾向
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive


In [59]:
# 例子：分析商品评价表露的情绪有哪些
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


happy, satisfied, grateful, impressed, content


In [60]:
# 例子：分析这段商品评价是否包含愤怒情绪
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


No


## 实体识别

如果我们需要提取出句子中提到的特定事物的名称，则可以采用类似如下例子的提示语：

In [61]:
# 例子：提取商品评价中提到的商品名和公司名

prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)




{
  "Item": "lamp",
  "Brand": "Lumina"
}


In [62]:
# 例子：同时进行情感分析与实体识别
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


## 主题判断

主题是一种比摘要更简洁直观地让我们了解文章的方式。一篇文章的摘要包含了其大部分主要内容，而主题只涉及文章内容属于哪些领域，讨论了哪些话题。通过创建自动推断文章主题的脚本，我们可以实现文章自动归类和新闻提醒功能。

In [63]:
# 例子：推断故事文本的主题
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

# 输出：政府调查、工作满意度、NASA、社会保障局、员工问题

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


In [64]:
# 例子：判断故事文本的主题是否在用户关注的话题列表中，如果是则提醒用户
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
""" # 这里要求模型直接生成JSON格式更规范！
response = get_completion(prompt)
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")



ALERT: New NASA story!


# 文本转换

将文本输入转换为不同的形式是大语言模型擅长的工作。例如，将一种语言的文本翻译为另一种语言，或对文本进行拼写或语法更正。文本转换甚至包括将文本转换为截然不同的形式，如 HTML 转 JSON，把一段描述转为正则表达式或 python 代码等等。

## 翻译任务

In [65]:
# 例子：将一段话从英文翻译到西班牙文
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)



Hola, me gustaría ordenar una licuadora.


In [66]:
# 例子：识别一段文本属于哪种语言
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)



This is French.


In [67]:
# 例子：将一段文本翻译为多种语言/方言
prompt = f"""
Translate the following  text to French language, to Spanish language,
and to English pirate language: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)


French: Je veux commander un ballon de basket
Spanish: Quiero ordenar una pelota de baloncesto
English Pirate: Arrr, I be wantin' to order a basketball, matey!


In [68]:
# 例子：将一段文本翻译为西班牙语，并且包括正式和不正式的用语场合
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)


Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


In [72]:
# 例子：将使用不同语言的商品评价一律翻译为英文和韩文
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")
    sleep(20)
    
    prompt = f"""
    Translate the following  text to English \
    and Korean: \`\`\`{issue}\`\`\`
    """
    response = get_completion(prompt)
    print(response, "\n")
    sleep(20)


Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that do not light up.
Korean: 내 모니터에는 불이 켜지지 않는 픽셀이 있습니다. 

Original message (This is Italian.): Il mio mouse non funziona


KeyboardInterrupt: 

## 语法检查任务

在我们在进行邮件往来，论坛发帖，润色英文文章的时候，可以利用 ChatGPT 很方便地进行拼写和语法检查，以确保自己的内容不存在语法错误。

In [73]:
# 例子：对若干段话进行拼写和语法检查

text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)
    sleep(20)


The girl with the black and white puppies has a ball.
No errors found.
It's going to be a long day. Does the car need its oil changed?
Their goes my freedom. There going to bring they're suitcases.

Corrected version: 
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check ChatGPT for spelling ability.


In [3]:
# 例子：对一段话进行语法检查（并利用 redlines 库标注修改的部分）
from redlines import Redlines
from IPython.display import Markdown

text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))



<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I got </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style="color:red;font-weight:700;text-decoration:line-through;">It's </span><span style="color:red;font-weight:700;">Additionally, it's </span>a bit small for what I paid for <span style="color:red;font-weight:700;text-decoration:line-through;">it though. </span><span style="color:red;font-weight:700;">it. </span>I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter.
</span><span style="color:red;font-weight:700;">daughter.</span>

## 语言风格转换


In [4]:
# 例子：将一句话转换为较为正式的商业邮件风格
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)


Dear Sir/Madam,

I am writing to bring to your attention a standing lamp that I believe may be of interest to you. Please find attached the specifications for your review.

Thank you for your time and consideration.

Sincerely,

Joe


In [5]:
# 例子：对一段话进行语法纠正，同时转换为特定格式

prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: \`\`\`{text}\`\`\`
"""
response = get_completion(prompt)
display(Markdown(response))


Title: A Soft and Cute Panda Plushie for All Ages

As an adult, I can attest that pandas are not just for kids. That's why I got this adorable panda plushie for my daughter's birthday, after she kept taking mine from my room. And let me tell you, it was a hit! She takes it everywhere with her, and it's super soft and cute.

However, I did notice that one of the ears is a bit lower than the other, which I don't think was designed to be asymmetrical. Despite this minor flaw, the plushie is still of high quality and worth the price.

Speaking of price, I do think it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. But, the fact that it arrived a day earlier than expected made up for it. I got to play with it myself before giving it to my daughter, and I have to say, it's hard to resist its charm.

Overall, if you're looking for a soft and cute panda plushie that can be enjoyed by all ages, this is a great option. Just be aware of its size and the possibility of asymmetry in the ears.

## 数据/文本格式转换

我们可以利用 ChatGPT 方便地将数据或文本转换为不同的格式。JSON、XML、YAML 之间的相互转换，或文本、HTML、Markdown 之间的转换，均可以由 ChatGPT 来完成。



In [6]:
# 例子：JSON 转 HTML
from IPython.display import HTML

data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
display(HTML(response))


Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


# 扩展文本

扩展是指输入一小段文本（例如一系列说明或主题列表）并让大型语言模型生成一段较长的文本（例如电子邮件或关于某个主题的文章）的任务。扩展除了可以帮我们写一些报告之外，还可以用于作为头脑风暴的伙伴帮助我们获取灵感。

下面的例子将介绍如何根据客户对商品的评价自动生成电子邮件回复，以及模型温度这一参数的用法。涉及的客户评价如下：


In [8]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""


## 例子：根据客户反响生成客服回复

In [9]:
# 例子：根据客户反响生成客服回复
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: \`\`\`{review}\`\`\`
Review sentiment: {sentiment}
"""
response = get_completion (prompt)
print (response)



Dear valued customer,

Thank you for taking the time to leave a review about your recent purchase. We apologize for any inconvenience caused by the price increase and the quality of the product not meeting your expectations. We understand your frustration and we would like to make it right. Please feel free to reach out to our customer service team for further assistance.

Thank you again for your feedback and for choosing our product. We hope to have the opportunity to serve you better in the future.

Best regards,

AI customer agent


## 模型温度参数的应用

温度（temperature）是 GPT 模型的一个重要参数，它意味着模型对不同输出可能性的探索程度或随机性的高度。较简而言之，设置低的温度值意味着模型针对同一输入的输出比较固定，而较高的温度则会使模型对同一个输入产生更加多样化的输出。

在需要输出固定可预测性强的场合我们最好将温度设为 0，而在需要输出具有创造性和多样性的场景下则应该尽量设置一个比较的温度，如 0.7。

在我们之前的例子中，为了结果有更好的可重复性，模型温度均被默认设为 0，接下来我们将探索更高的温度对模型输出带来的影响。



In [10]:
# 例子：将温度设为 0.7 生成更多样化的客服回复

prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: \`\`\`{review}\`\`\`
Review sentiment: {sentiment}
"""
response = get_completion (prompt, temperature=0.7)
print (response)



Dear valued customer,

Thank you for taking the time to leave a review about your recent purchase. We are sorry to hear that you have experienced some issues with the price and quality of the product. We apologize for any inconvenience this may have caused you.

If you would like to discuss this further, please do not hesitate to reach out to our customer service team who will be more than happy to assist you. We value your feedback and appreciate your business.

Thank you again for your review and we hope to have the opportunity to serve you again in the future.

Best regards,
AI customer agent
